In [1]:
import pandas as pd
from bokeh.plotting import figure 
from bokeh.io import output_file, save, show
from bokeh.models import ColumnDataSource, HoverTool, FactorRange, NumeralTickFormatter
from bokeh.layouts import gridplot
from bokeh.transform import factor_cmap
from bokeh.models.annotations import Label
from bokeh.palettes import Blues3

In [2]:
from IPython.display import HTML, display

def set_css():
  display(HTML('''
  <style>
    pre {
        white-space: pre-wrap;
    }
  </style>
  '''))
get_ipython().events.register('pre_run_cell', set_css)

# Task 1: Prepare the Data

## 1.1: Use pandas to read a csv file into a dataframe from a link
You may refer to:  https://pythoninoffice.com/how-to-read-google-sheet-into-pandas/

In [3]:
url = 'https://docs.google.com/spreadsheets/d/e/2PACX-1vQdNgN-88U31tk1yQytaJdmoLrxuFn1LnbwTubwCd8se2aHh8656xLLHzxHSoiaXMUu8rIcu6gMj5Oq/pub?gid=1242961990&single=true&output=csv'
MAGMA_financials = pd.read_csv( url )
MAGMA_financials

,Symbol,Quarter Ended,Revenue,Cost of Revenue,Gross Profit,"Selling, General & Admin",Research & Development,Operating Expenses,Operating Income,Net Income
0,META,2019Q1,15077,2816,12261,6084,2860,8944,3317,2429
1,META,2019Q2,16886,3307,13579,5638,3315,8953,4626,2616
2,META,2019Q3,17652,3155,14497,3764,3548,7312,7185,6091
3,META,2019Q4,21082,3491,17591,4855,3878,8733,8858,7349
4,META,2020Q1,17737,3459,14278,4370,4015,8385,5893,4902
...,...,...,...,...,...,...,...,...,...,...
75,AMZN,2021Q4,137412,82835,54577,13335,15313,51117,3460,14323
76,AMZN,2022Q1,116444,66499,49945,10914,14842,46276,3669,-3844
77,AMZN,2022Q2,121234,66424,54810,12989,18072,51493,3317,-2028
78,AMZN,2022Q3,127101,70268,56833,14075,19485,54308,2525,2872


## 1.2: Inspect the columns in the dataframe
- 'Symbol':  The stock ticker symbols of five companies -> META, AAPL, GOOGL, MSFT, AMZN 

- 'Quarter Ended': The time when the companies filed their quarterly financial statements to SEC -> from 2019Q1 to 2022Q4

- The other columns, 'Revenue', 'Gross Profit', etc.: are several items in the financial statements

Your task is to select a subset of the financial items of a company and visualize them in a grouped bar chart
Select a subset of the financial items, e.g. ['Net Income', 'Operating Expenses', 'Selling, General & Admin']

In [4]:
subset = MAGMA_financials[["Quarter Ended","Net Income", "Revenue", "Cost of Revenue"]]
subset.head()

,Quarter Ended,Net Income,Revenue,Cost of Revenue
0,2019Q1,2429,15077,2816
1,2019Q2,2616,16886,3307
2,2019Q3,6091,17652,3155
3,2019Q4,7349,21082,3491
4,2020Q1,4902,17737,3459


## 1.3: Create a nested categorical coordinate

 To see the trend, you'll use 'Quarter Ended' in the x axis
 Each item has 16 quarters' data, and each quarter has a subset of items
 To make the x axis clear and readable, 
 you'll create a nested (hierarchical) categorical coordinate as x
 Namely, it will group the bars in nested levels of (year, quarter, item)
 e.g. x = [ ('2019', 'Q1', 'item 1'),  ('2019', 'Q1', 'item 2'), ...]

 You may refer to:
 bokeh tutorial 07 - Bar and Categorical Data Plots - Grouped Bar Charts
